<a href="https://colab.research.google.com/github/DucBox/CyberSecurity/blob/main/Salsa20_%2B_DES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def rotl(value, shift, bits=32):
    """Rotate left a 32-bit integer value by a specified shift.

    Args:
        value: The integer to rotate.
        shift: The number of bits by which to rotate.
        bits: The size of the integer, in bits (default is 32).

    Returns:
        The rotated integer.
    """
    return ((value << shift) | (value >> (bits - shift))) & ((1 << bits) - 1)

def quarter_round(a, b, c, d):
    """Perform a quarter round operation on four integers.

    Args:
        a, b, c, d: Four 32-bit integers.

    Returns:
        The four integers after the quarter round operation.
    """
    a = (a + b) & 0xffffffff; d ^= a; d = rotl(d, 16)
    c = (c + d) & 0xffffffff; b ^= c; b = rotl(b, 12)
    a = (a + b) & 0xffffffff; d ^= a; d = rotl(d, 8)
    c = (c + d) & 0xffffffff; b ^= c; b = rotl(b, 7)
    return a, b, c, d

def salsa20_hash(state):
    """Compute the Salsa20 hash function on a 512-bit state.

    Args:
        state: A list of 16 integers, the 512-bit state.

    Returns:
        A list of 16 integers after 20 rounds of Salsa20 operation.
    """
    x = list(state)
    for _ in range(10):  # 20 rounds, performed in pairs
        # Column rounds
        x[4], x[8], x[12], x[0] = quarter_round(x[4], x[8], x[12], x[0])
        x[9], x[13], x[1], x[5] = quarter_round(x[9], x[13], x[1], x[5])
        x[14], x[2], x[6], x[10] = quarter_round(x[14], x[2], x[6], x[10])
        x[3], x[7], x[11], x[15] = quarter_round(x[3], x[7], x[11], x[15])
        # Diagonal rounds
        x[1], x[2], x[3], x[0] = quarter_round(x[1], x[2], x[3], x[0])
        x[6], x[7], x[4], x[5] = quarter_round(x[6], x[7], x[4], x[5])
        x[11], x[8], x[9], x[10] = quarter_round(x[11], x[8], x[9], x[10])
        x[12], x[13], x[14], x[15] = quarter_round(x[12], x[13], x[14], x[15])
    return [(x[i] + state[i]) & 0xffffffff for i in range(16)]

def salsa20_key_schedule(key, nonce, counter):
    """Generate the Salsa20 key schedule.

    Args:
        key: The encryption key as a byte string.
        nonce: The nonce as a byte string.
        counter: The block counter.

    Returns:
        A list of 16 integers that represent the key schedule.
    """
    constants = [0x61707865, 0x3320646e, 0x79622d32, 0x6b206574]
    key_schedule = constants[:1] + [int.from_bytes(key[i:i+4], 'little') for i in range(0, 16, 4)] \
                  + constants[1:3] + [counter & 0xffffffff, counter >> 32] \
                  + [int.from_bytes(nonce[i:i+4], 'little') for i in range(0, 8, 4)] + constants[3:]
    if len(key) == 32:
        key_schedule[1:5] += [int.from_bytes(key[i:i+4], 'little') for i in range(16, 32, 4)]
    elif len(key) != 16:
        raise ValueError("Key must be either 16 or 32 bytes long.")
    return key_schedule
def encrypt(plaintext, key, nonce):
    """Encrypt plaintext using the Salsa20 algorithm.

    Args:
        plaintext: The plaintext message as a byte string.
        key: The encryption key as a byte string (must be 16 or 32 bytes).
        nonce: The nonce as a byte string (must be 8 bytes).

    Returns:
        The encrypted message as a byte string.
    """
    assert len(nonce) == 8, "Nonce must be 8 bytes long"
    encrypted = b''
    counter = 0
    while plaintext:
        block = plaintext[:64]
        plaintext = plaintext[64:]
        key_schedule = salsa20_key_schedule(key, nonce, counter)
        keystream = salsa20_hash(key_schedule)
        keystream_bytes = b''.join(word.to_bytes(4, 'little') for word in keystream)
        encrypted_block = bytes(a ^ b for a, b in zip(block, keystream_bytes[:len(block)]))
        encrypted += encrypted_block
        counter += 1
    return encrypted

def decrypt(ciphertext, key, nonce):
    """Decrypt ciphertext using the Salsa20 algorithm.

    This function is identical to the encrypt function due to the
    symmetric nature of the algorithm.

    Args:
        ciphertext: The encrypted message as a byte string.
        key: The encryption key as a byte string (must be 16 or 32 bytes).
        nonce: The nonce as a byte string (must be 8 bytes).

    Returns:
        The decrypted message as a byte string.
    """
    return encrypt(ciphertext, key, nonce)

# Example usage
key = b"this_is_a_32_byte_key_for_salsa!"
nonce = b"nonce123"
plaintext = b"Hello, Salsa20!"

encrypted = encrypt(plaintext, key, nonce)
print(f"Encrypted: {encrypted.hex()}")

decrypted = decrypt(encrypted, key, nonce)
print(f"Decrypted: {decrypted.decode()}")



Encrypted: 0109fc750bc2b016264d845aedb744
Decrypted: Hello, Salsa20!


In [ ]:
def KSA(key):
    key_length = len(key)
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % key_length]) % 256
        S[i], S[j] = S[j], S[i]
    return S

def PRGA(S):
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        S[i], S[j] = S[j], S[i]
        K = S[(S[i] + S[j]) % 256]
        yield K

def RC4(key):
    S = KSA(key)
    return PRGA(S)


In [ ]:
def benchmark_stream_generation(cipher_func, key, nonce=None, data_size=1024*1024):
    start_time = time.time()
    # Generate a block of data
    data = b'\x00' * data_size
    if nonce:  # For Salsa20
        _ = cipher_func(data, key, nonce)
    else:  # For RC4, assuming cipher_func is adjusted to return bytes
        _ = cipher_func(data, key)
    end_time = time.time()
    return end_time - start_time

# Adjust RC4 to return bytes instead of a generator
def RC4_to_bytes(key, data):
    keystream = RC4(key)
    return bytes([next(keystream) for _ in range(len(data))])

# Benchmark Salsa20 and RC4
salsa20_time = benchmark_stream_generation(encrypt, key, nonce)
rc4_time = benchmark_stream_generation(RC4_to_bytes, key)

print(f"Salsa20 Generation Time: {salsa20_time} seconds")
print(f"RC4 Generation Time: {rc4_time} seconds")


Salsa20 Generation Time: 5.387058973312378 seconds
RC4 Generation Time: 0.0003523826599121094 seconds


In [ ]:
def key_sensitivity_test(encrypt_function, key, nonce=None):
    data = b'\x00' * 512  # Dùng một khối dữ liệu nhỏ để mã hóa
    flipped_key = bytearray(key)
    flipped_key[0] ^= 1  # Flip the first bit of the key

    encrypted1 = encrypt_function(data, key, nonce) if nonce else RC4_to_bytes(key, data)
    encrypted2 = encrypt_function(data, bytes(flipped_key), nonce) if nonce else RC4_to_bytes(bytes(flipped_key), data)

    # Calculate the number of differing bytes
    differences = sum(1 for x, y in zip(encrypted1, encrypted2) if x != y)
    return differences

# So sánh độ nhạy khóa
difference_salsa20 = key_sensitivity_test(encrypt, key, nonce)
difference_rc4 = key_sensitivity_test(RC4_to_bytes, key)

print(f"Salsa20 Key Sensitivity: {difference_salsa20} bytes differ")
print(f"RC4 Key Sensitivity: {difference_rc4} bytes differ")


Salsa20 Key Sensitivity: 511 bytes differ
RC4 Key Sensitivity: 510 bytes differ


In [ ]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.9 MB/s eta 0:00:00


In [ ]:
from Crypto.Cipher import DES, ARC4
from Crypto.Random import get_random_bytes
import time

# Padding để đảm bảo dữ liệu có độ dài là bội số của DES block size
def pad(text):
    while len(text) % 8 != 0:
        text += b' '
    return text

# Hàm mã hóa DES
def encrypt_des(plaintext, key):
    des = DES.new(key, DES.MODE_ECB)
    padded_text = pad(plaintext)
    encrypted_text = des.encrypt(padded_text)
    return encrypted_text

# Hàm giải mã DES
def decrypt_des(ciphertext, key):
    des = DES.new(key, DES.MODE_ECB)
    decrypted_text = des.decrypt(ciphertext)
    return decrypted_text.strip()

# Hàm mã hóa RC4
def encrypt_rc4(plaintext, key):
    rc4 = ARC4.new(key)
    encrypted_text = rc4.encrypt(plaintext)
    return encrypted_text

# Hàm giải mã RC4
def decrypt_rc4(ciphertext, key):
    rc4 = ARC4.new(key)
    decrypted_text = rc4.decrypt(ciphertext)
    return decrypted_text

# Đo lường thời gian
def measure_time(func, *args):
    start_time = time.time()
    func(*args)
    end_time = time.time()
    return end_time - start_time

# Tạo key ngẫu nhiên cho DES và RC4
key_des = get_random_bytes(8)
key_rc4 = get_random_bytes (16)

In [ ]:

# Tạo các case đa dạng để test
cases = [
    b"Short text.",
    b"https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4",
    b"https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4 https://colab.research.google.com/drive/148CgeQQzID6h9XUVg9Igz2wtqXeK6Hin#scrollTo=uGKTjVMb1EB4 It contains more characters and therefore, it is expected to take a longer time to encrypt and decrypt compared to shorter texts.",
    b"The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog The quick brown fox jumps over the lazy dog"
]

# Đo lường thời gian mã hóa cho mỗi case
for i, data in enumerate(cases, 1):
    time_des = measure_time(encrypt_des, data, key_des)
    time_rc4 = measure_time(encrypt_rc4, data, key_rc4)
    print(f'Case {i}:')
    print(f'Time taken for DES encryption: {time_des:.6f} seconds')
    print(f'Time taken for RC4 encryption: {time_rc4:.6f} seconds\n')


Case 1:
Time taken for DES encryption: 0.001227 seconds
Time taken for RC4 encryption: 0.000046 seconds

Case 2:
Time taken for DES encryption: 0.000088 seconds
Time taken for RC4 encryption: 0.000043 seconds

Case 3:
Time taken for DES encryption: 0.000096 seconds
Time taken for RC4 encryption: 0.000045 seconds

Case 4:
Time taken for DES encryption: 0.000054 seconds
Time taken for RC4 encryption: 0.000035 seconds



In [ ]:
# Tạo các loại dữ liệu khác nhau để làm case test
binary_data = get_random_bytes(256)  # Dữ liệu nhị phân ngẫu nhiên
numeric_data = bytes([i for i in range(256)])  # Dãy số từ 0 đến 255
repeated_data = b"A" * 256  # Dữ liệu là ký tự lặp đi lặp lại
mixed_data = b"".join([bytes([i, 65]) for i in range(128)])  # Kết hợp giữa số và chữ

# Thêm các case dữ liệu mới vào danh sách
new_cases = [
    ("Binary Data", binary_data),
    ("Numeric Data", numeric_data),
    ("Repeated Character", repeated_data),
    ("Mixed Data", mixed_data),
]

# Đo lường thời gian mã hóa cho mỗi loại dữ liệu mới
for name, data in new_cases:
    time_des = measure_time(encrypt_des, data, key_des)
    time_rc4 = measure_time(encrypt_rc4, data, key_rc4)
    print(f'{name}:')
    print(f'Time taken for DES encryption: {time_des:.6f} seconds')
    print(f'Time taken for RC4 encryption: {time_rc4:.6f} seconds\n')


Binary Data:
Time taken for DES encryption: 0.000157 seconds
Time taken for RC4 encryption: 0.000043 seconds

Numeric Data:
Time taken for DES encryption: 0.000127 seconds
Time taken for RC4 encryption: 0.000034 seconds

Repeated Character:
Time taken for DES encryption: 0.000120 seconds
Time taken for RC4 encryption: 0.000036 seconds

Mixed Data:
Time taken for DES encryption: 0.000134 seconds
Time taken for RC4 encryption: 0.000036 seconds



In [15]:
# DES implementation

# Initial Permutation Table
initial_permutation_table = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9, 1,
    59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

# Final Permutation Table
final_permutation_table = [
    40, 8, 48, 16, 56, 24, 64, 32,
    39, 7, 47, 15, 55, 23, 63, 31,
    38, 6, 46, 14, 54, 22, 62, 30,
    37, 5, 45, 13, 53, 21, 61, 29,
    36, 4, 44, 12, 52, 20, 60, 28,
    35, 3, 43, 11, 51, 19, 59, 27,
    34, 2, 42, 10, 50, 18, 58, 26,
    33, 1, 41, 9, 49, 17, 57, 25
]

# Initial Permutation
def initial_permutation(block):
    return permute(block, initial_permutation_table, 64)

# Final Permutation
def final_permutation(block):
    return permute(block, final_permutation_table, 64)

# Permutation function
def permute(block, table, block_size):
    permuted_block = 0
    for index, bit_index in enumerate(table):
        shift_count = block_size - bit_index
        if shift_count < 0 or block_size - index - 1 < 0:
            continue  # Skip if shift count is negative or index is out of range
        bit = (block >> shift_count) & 1
        permuted_block |= bit << (block_size - index - 1)
    return permuted_block


# Expansion Permutation Table
expansion_permutation_table = [
    32, 1, 2, 3, 4, 5,
    4, 5, 6, 7, 8, 9,
    8, 9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32, 1
]

# S-boxes (Substitution Boxes)
s_boxes = [
    [
        [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
        [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
        [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
        [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
    ],
    # S2
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],
    # S3
    [
        [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
        [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
        [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
        [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]
    ],
    # S4
    [
        [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
        [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
        [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
        [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]
    ],

    # S5
    [
        [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
        [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
        [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
        [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]
    ],
    # S6
    [
        [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
        [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
        [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
        [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]
    ],
    # S7
    [
        [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
        [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
        [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
        [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]
    ],
    # S8
    [
        [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
        [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
        [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
        [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
    ]
]

# Straight Permutation Table
straight_permutation_table = [
    16, 7, 20, 21, 29, 12, 28, 17,
    1, 15, 23, 26, 5, 18, 31, 10,
    2, 8, 24, 14, 32, 27, 3, 9,
    19, 13, 30, 6, 22, 11, 4, 25
]

# Key Permutation Table
key_permutation_table = [
    57, 49, 41, 33, 25, 17, 9, 1,
    58, 50, 42, 34, 26, 18, 10, 2,
    59, 51, 43, 35, 27, 19, 11, 3,
    60, 52, 44, 36, 63, 55, 47, 39,
    31, 23, 15, 7, 62, 54, 46, 38,
    30, 22, 14, 6, 61, 53, 45, 37,
    29, 21, 13, 5, 28, 20, 12, 4
]

# Key Compression Table
key_compression_table = [
    14, 17, 11, 24, 1, 5, 3, 28,
    15, 6, 21, 10, 23, 19, 12, 4,
    26, 8, 16, 7, 27, 20, 13, 2,
    41, 52, 31, 37, 47, 55, 30, 40,
    51, 45, 33, 48, 44, 49, 39, 56,
    34, 53, 46, 42, 50, 36, 29, 32
]

# Padding for keys less than 64 bits
# Padding for keys less than 64 bits
def pad_key(key):
    key = str(key)  # Convert integer key to string
    while len(key) < 8:
        key += " "
    return key


# Key shift values
key_shifts = [
    1, 1, 2, 2,
    2, 2, 2, 2,
    1, 2, 2, 2,
    2, 2, 2, 1
]

# Define left and right masks
left_mask = (1 << 28) - 1
right_mask = (1 << 28) - 1

# Key Generation
def generate_round_keys(key):
    key = pad_key(key)
    key = permute(int.from_bytes(key.encode(), byteorder='big'), key_permutation_table, 64)
    round_keys = []
    # Splitting key into two parts
    left = (key >> 28) & ((1 << 28) - 1)
    right = key & ((1 << 28) - 1)
    for i in range(16):
        # Shifting bits
        left = ((left << key_shifts[i]) & left_mask) | ((left >> (28 - key_shifts[i])) & right_mask)
        right = ((        right << key_shifts[i]) & left_mask) | ((right >> (28 - key_shifts[i])) & right_mask)
        # Combining the two halves
        combined_key = (left << 28) | right
        # Key Compression
        round_key = permute(combined_key, key_compression_table, 56)
        round_keys.append(round_key)
    return round_keys

# DES round function
def des_round(block, round_key):
    # Expansion Permutation
    expanded_block = permute(block, expansion_permutation_table, 32)
    # XOR with Round Key
    xor_result = expanded_block ^ round_key
    # Substitution with S-boxes
    substitution_result = 0
    for i in range(8):
        # Extracting 6 bits for each S-box
        bits = (xor_result >> (6 * (7 - i))) & 0b111111
        # Finding row and column indices
        row = ((bits & 0b100000) >> 4) | (bits & 0b1)
        col = (bits >> 1) & 0b1111
        # Substituting with S-boxes
        substitution_result |= s_boxes[i][row][col] << (4 * (7 - i))
    # Straight Permutation
    straight_permuted_result = permute(substitution_result, straight_permutation_table, 32)
    return straight_permuted_result

# DES Encryption
def des_encryption(plain_text, key):
    round_keys = generate_round_keys(key)
    plain_text = initial_permutation(plain_text)
    left_half = plain_text >> 32
    right_half = plain_text & 0xFFFFFFFF
    for i in range(16):
        temp = right_half
        right_half = left_half ^ des_round(right_half, round_keys[i])
        left_half = temp
    # Final Permutation
    cipher_text = (right_half << 32) | left_half
    return final_permutation(cipher_text)

# RC4 implementation

# Key-Scheduling Algorithm (KSA)
def rc4_key_scheduling_algorithm(key):
    key_length = len(key)
    # Initialization of S array
    S = list(range(256))
    j = 0
    for i in range(256):
        j = (j + S[i] + key[i % key_length]) % 256
        # Swap S[i] and S[j]
        S[i], S[j] = S[j], S[i]
    return S

# Pseudo-Random Generation Algorithm (PRGA)
def rc4_pseudo_random_generation_algorithm(plaintext, key):
    S = rc4_key_scheduling_algorithm(key)
    i = 0
    j = 0
    cipher_text = bytearray()
    for char in plaintext:
        i = (i + 1) % 256
        j = (j + S[i]) % 256
        # Swap S[i] and S[j]
        S[i], S[j] = S[j], S[i]
        # Generate keystream
        keystream = S[(S[i] + S[j]) % 256]
        # XOR with plaintext to get ciphertext
        cipher_text.append(char ^ keystream)
    return bytes(cipher_text)

# DES Test
plaintext = 0x0123456789ABCDEF
key = 0x133457799BBCDFF1
cipher_text_des = des_encryption(plaintext, key)
print("Cipher Text (DES):", hex(cipher_text_des))

# RC4 Test
plaintext = b"Hello, World!"
key = b"SecretKey"
cipher_text_rc4 = rc4_pseudo_random_generation_algorithm(plaintext, key)
print("Cipher Text (RC4):", cipher_text_rc4.hex())


Cipher Text (DES): 0x41db1c2ad145f52f
Cipher Text (RC4): 5cdd502bb05e3ef58b6151228c


In [10]:
text_content = """dfiushfodsfhd9foadjfodsfaff"""
file_path = 'small_text.txt'

with open(file_path, 'w') as f:
    f.write(text_content)


In [9]:
import os

file_path = 'random_file.bin'
file_size = 1024 * 1024  # 1MB

with open(file_path, 'wb') as f:
    f.write(os.urandom(file_size))


In [22]:
import time
des_key = 0x133457799BBCDFF1
rc4_key = b"SecretKey"
plaintext_files = ["small_text.txt", "random_file.bin", "bg.png"]
def read_file_content(file_path):
    with open(file_path, "rb") as file:
        return file.read()
# Re-defining the encryption wrapper for DES to handle arbitrary file sizes based on the provided DES implementation
def des_encrypt_file_content(content, key):
    block_size = 8  # DES works on 64 bit (8 byte) blocks
    if len(content) % block_size != 0:
        # Add padding if content is not a multiple of 8 bytes
        padding_length = block_size - (len(content) % block_size)
        content += bytes([padding_length] * padding_length)
    encrypted_content = b''
    for i in range(0, len(content), block_size):
        block = int.from_bytes(content[i:i+block_size], byteorder='big')
        encrypted_block = des_encryption(block, key)
        encrypted_content += encrypted_block.to_bytes(block_size, byteorder='big')
    return encrypted_content

def rc4_encrypt_file_content(content, key):
    # Encrypting content using RC4. This function already handles arbitrary lengths.
    return rc4_pseudo_random_generation_algorithm(content, key)

# Performance comparison with the corrected DES encryption function
performance_results_corrected = []

for file_path in plaintext_files:
    content = read_file_content(file_path)

    # DES encryption
    start_time = time.time()
    des_encrypted_content = des_encrypt_file_content(content, des_key)
    des_time = time.time() - start_time

    # RC4 encryption
    start_time = time.time()
    rc4_encrypted_content = rc4_encrypt_file_content(content, rc4_key)
    rc4_time = time.time() - start_time

    # Store results
    performance_results_corrected.append({
        'file': os.path.basename(file_path),
        'des_time': des_time,
        'rc4_time': rc4_time,
        'des_size': len(des_encrypted_content),
        'rc4_size': len(rc4_encrypted_content)
    })

performance_results_corrected


[{'file': 'small_text.txt',
  'des_time': 0.009921073913574219,
  'rc4_time': 0.00015664100646972656,
  'des_size': 32,
  'rc4_size': 27},
 {'file': 'random_file.bin',
  'des_time': 139.89867544174194,
  'rc4_time': 0.4503054618835449,
  'des_size': 1048576,
  'rc4_size': 1048576},
 {'file': 'bg.png',
  'des_time': 81.46884775161743,
  'rc4_time': 0.27041029930114746,
  'des_size': 634432,
  'rc4_size': 634428}]